# Compaction Project

## Problem Formulation

Given a set of chunks whose sizes are

$$
S_n = \{d_1, \cdots, d_n\},
$$

where the positive integer $d_i \leq 2048$ for all $i = 1, \cdots, n$. Suppose all join operators need time $f(d_i)$ to process a data chunk with the size $d_i$. 

Our goal is to compact the set $S$, i.e., we need a transformation

$$
\mathcal{M}: S_n \rightarrow S'_m \triangleq \{d'_1,  \cdots, d'_m\},
$$

where $\sum_i^n d_i = \sum_j^m d'_j$ and $m$ is an arbitrary integer less than $n$, to minimize 

$$
\sum_j^m f(d'_j) + cost(M, S).
$$

where $cost(\mathcal{M}, S)$ is the cost of the transformation $\mathcal{M}$ on the set $S$. The cost of combining two or more chunks into one: $d_i + \cdots + d_j = d'_s \leq 2048$, is 

$$
g(d'_s) = C_3 + d'_s \times C_4.
$$

**Note:** This formulated problem is easier than the real compaction problem because we have the sizes of all chunks in advance, rather than a chunk stream.

In [ ]:
# import
from termcolor import colored
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# settings
%matplotlib inline

In [ ]:
# utils
def print_color(text, color='black'):
    print(colored(text, color))

## 1. Chunk Sizes Distribution

In [ ]:
# Generate random chunk sizes from a Gaussian distribution
def generate_chunk_sizes(n, mean=64, scale=256):
    return np.minimum(2048, np.maximum(1, np.random.normal(mean, scale, n))).astype(int)

## 2. Compaction Simulator

In [ ]:
#             fixed cost      per tuple cost
# probe()     1.5             0.03
# next()      0.9             0.06
# --------------------------------------
# compact()   0.3             0.03
# --------------------------------------

k_prc_fixed_cost = (1.5 + 0.9)
k_prc_per_tuple_cost = (0.03 + 0.06)
k_cpt_fixed_cost = 0.3
k_cpt_per_tuple_cost = 0.03

# This function split each chunk into smaller chunks.
def split_array_np(numbers, parts):
    numbers = np.array(numbers)

    quotient, remainder = np.divmod(numbers, parts)
    split = np.repeat(quotient, parts)
    remainder = np.repeat(remainder, parts)
    split[np.arange(numbers.size * parts) % parts < remainder] += 1
        
    split = split[split > 0]

    return split

# Simulate the execution of joins.
def simulate_join(sizes, compact_func, chunk_factor=1, n_join=1, print_log=True):
    prc_cost = 0
    cpt_cost = 0
    next_sizes = np.array(sizes)

    if print_log:
        print_color(f"-------------------------", 'green')
        print_color(f"Compactor {compact_func}", 'green')

    for l in reversed(range(n_join)):
        # 1. Join
        prc_cost += k_prc_fixed_cost * len(next_sizes) + np.sum(next_sizes) * k_prc_per_tuple_cost

        # 2. Split chunks
        next_sizes = split_array_np(next_sizes, chunk_factor)
        input_chunk_number = len(next_sizes)

        # 3. Compact
        next_sizes, cost = compact_func(next_sizes, chunk_factor, l)
        output_chunk_number = len(next_sizes)

        if print_log:
            print_color(f"Level {n_join - l}: {input_chunk_number} -> {output_chunk_number} chunks, cost: {cost:.2f}", 'green')

        cpt_cost += cost

    return prc_cost, cpt_cost

## 3. Compaction Strategies

### Cost Calculation

Suppose the $i$-th join operator needs time $f_1(d) = C_1 + C_2 \cdot d$ to process a data chunk with the size $d$. 

We also suppose the pipeline has $n$ join operators, each join receives a data chunk, and outputs $m$ smaller chunks. 

Note that the total number of tuples across these smaller chunks remains the same as the number of tuples in the original input chunk. 

Then, for an input chunk with the size $d$, the time needed to process consists of two parts: 

1. Per Tuple Cost 

$$ C_1 \cdot d \cdot n $$ 

2. Fixed Cost

$$ C_2 \cdot \min\{m^0, d\} + C_2 \cdot \min\{m^1, d\} + \cdots + C_2 \cdot \min\{m^{n-1}, d\}$$

We need to take the minimum value betwwen $m^{i}$ and the $d$ because a data chunk cannot be split into more than $d$ smaller chunks. In other words, each chunk has at least one tuple.

In [ ]:
def compute_prc_cost(chunk_size, chunk_factor, n_joins):
    per_tuple_cost = k_prc_per_tuple_cost * chunk_size * n_joins

    fixed_cost = 0
    for i in range(n_joins):
        fixed_cost += k_prc_fixed_cost * np.minimum(chunk_factor ** i, chunk_size)
    
    # print(f"per_tuple_cost: {per_tuple_cost:.2f}, fixed_cost: {fixed_cost:.2f}")
    return per_tuple_cost + fixed_cost


# Compute the cost of a single compaction
def compute_cpt_cost(sizes_in_one_compaction):
    return k_cpt_fixed_cost + np.sum(sizes_in_one_compaction) * k_cpt_per_tuple_cost

### Base Strategies

**Strategy 1**: Do not compact any chunks. 

**Strategy 2**: Fully compact all chunks.

In [ ]:
def alg_no_compaction(chunk_sizes, chunk_factor, level):
    return chunk_sizes, 0


def alg_full_compaction(chunk_sizes, chunk_factor, level):
    transformed_sizes = []
    cpt_cost = 0
    cpt_sizes = []
    
    for size in chunk_sizes:
        if size == 2048: 
            transformed_sizes.append(size)
            continue

        if sum(cpt_sizes) + size <= 2048:
            cpt_sizes.append(size)
        else:
            cpt_cost += compute_cpt_cost(cpt_sizes)
            transformed_sizes.append(sum(cpt_sizes))
            cpt_sizes = [size]
    
    if cpt_sizes:
        cpt_cost += compute_cpt_cost(cpt_sizes)
        transformed_sizes.append(sum(cpt_sizes))

    return transformed_sizes, cpt_cost

### Optimal Strategy

**Strategy 3**: Sort all chunks ascendingly and compact them in order, until compaction is not beneficial

In [ ]:
def alg_sort_compaction(chunk_sizes, chunk_factor, n_joins):
    assert len(chunk_sizes) > 0, "chunk_sizes must not be empty"

    sorted_sizes = sorted(chunk_sizes)
    transformed_sizes = []
    cpt_cost = 0

    i = 0
    cpt_sizes = [sorted_sizes[0]]
    for i in range(1, len(sorted_sizes), 1):
        size = sorted_sizes[i]
        cur_sum = sum(cpt_sizes)

        if cur_sum + size <= 2048:
            gain = compute_prc_cost(cur_sum, chunk_factor, n_joins) + compute_prc_cost(size, chunk_factor, n_joins) - compute_prc_cost(cur_sum + size, chunk_factor, n_joins)
            loss = (k_cpt_fixed_cost + size * k_cpt_per_tuple_cost)
            
            # print(f"gain: {gain:.2f}, loss: {loss:.2f}")
            # print(f"cur sum {cur_sum} + size {size} = {cur_sum + size}")

            if gain - loss > 0:
                cpt_sizes.append(size)
            else:
                break
        else:
            if len(cpt_sizes) > 1:
                cpt_cost += compute_cpt_cost(cpt_sizes)
            transformed_sizes.append(sum(cpt_sizes))

            cpt_sizes = [size]

    if cpt_sizes:
        if len(cpt_sizes) > 1:
            cpt_cost += compute_cpt_cost(cpt_sizes)
        transformed_sizes.append(sum(cpt_sizes))

    # Append the remaining chunks, it is not beneficial to compact them
    for j in range(i+1, len(sorted_sizes)):
        transformed_sizes.append(sorted_sizes[j])
        
    return transformed_sizes, cpt_cost

In [ ]:
def compute_benefit(buffer, target, chunk_factor, n_joins):
    prc_gain = compute_prc_cost(buffer, chunk_factor, n_joins) + compute_prc_cost(target, chunk_factor, n_joins) - compute_prc_cost(buffer + target, chunk_factor, n_joins)
    cpt_cost = (k_cpt_fixed_cost + target * k_cpt_per_tuple_cost)
    return prc_gain - cpt_cost

chunk_factor = 4
n_joins = 2
x_base = np.arange(1, 2048, 1)
y_add = np.arange(1, 2048, 1)

z_benefits = np.zeros((2047, 2047))
for i in range(2047):
    for j in range(2047):
        if x_base[i] + y_add[j] <= 2048 and x_base[i] + y_add[j] > 0:
            z_benefits[j][i] = compute_benefit(x_base[i], y_add[j], chunk_factor, n_joins)
        else:
            z_benefits[j][i] = None

In [ ]:
# Assuming z_benefits is a numpy array
y_zero, x_zero = np.where(np.abs(z_benefits) < 0.01)

fig = go.Figure(data=go.Heatmap(x=x_base, y=y_add, z=z_benefits, colorscale='ice', showlegend=False))

fig.add_trace(go.Scatter(x=x_base[x_zero], y=y_add[y_zero], mode='lines', name='z = 0', line=dict(color='black',width=2)))

fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    title="Compaction Benefit",
    xaxis_title="Buffer Chunk Size",
    yaxis_title="Target Chunk Size",
)

fig.show()

### DuckDB Strategy

**Strategy 4**: Set a threhold to distinguish the big chunk and the small chunk, we only compact small chunks.

In [ ]:
k_block_size = 2048
k_duckdb_compaction_threshold = 128

def alg_binary_compaction(chunk_sizes, chunk_factor, level):
    trans_chunks = []
    cpt_cost = 0
    
    cpt_chunks = []
    for size in chunk_sizes:
        if size >= k_duckdb_compaction_threshold:
            trans_chunks.append(size)
            continue

        cpt_chunks.append(size)

        if sum(cpt_chunks) >= k_block_size - k_duckdb_compaction_threshold:
            trans_chunks.append(np.sum(cpt_chunks))
            cpt_cost += compute_cpt_cost(cpt_chunks)
            cpt_chunks = []
    
    return trans_chunks, cpt_cost

**Strategy 5**: The last strategy uses fixed threshold 128 for all levels. It is not precise, and we can compute a better one 

In [ ]:
def get_cpt_threhold(chunk_factor, n_joins):
    if n_joins == 0:
        return 0

    x = 1024
    y = np.arange(1, 2048, 1)
    z = np.zeros((2047))

    for i in range(2047):
        if x + y[i] <= 2048 and x + y[i] > 0:
            z[i] = compute_benefit(x, y[i], chunk_factor, n_joins)
        else:
            z[i] = None

    y_zero = np.where(np.abs(z) < 0.01)

    return y_zero[0][0] if len(y_zero[0]) > 0 else 1024

In [ ]:
def alg_dynamic_compaction(chunk_sizes, chunk_factor, level):
    cpt_threshold = get_cpt_threhold(chunk_factor, level)

    print_color(f"Compaction Threshold: {cpt_threshold}", 'blue')

    trans_chunks = []
    cpt_cost = 0
    
    cpt_chunks = []
    for size in chunk_sizes:
        if size >= cpt_threshold:
            trans_chunks.append(size)
            continue

        cpt_chunks.append(size)

        if sum(cpt_chunks) >= k_block_size - cpt_threshold:
            trans_chunks.append(np.sum(cpt_chunks))
            cpt_cost += compute_cpt_cost(cpt_chunks)
            cpt_chunks = []

    
    return trans_chunks, cpt_cost

## 4. Let's Join!

In [ ]:
def let_us_join(chunk_sizes, chunk_factor, num_join, print_log=True):
    grades = {
        "No Compaction": simulate_join(chunk_sizes, alg_no_compaction, chunk_factor, num_join, print_log), 
        "Full Compaction": simulate_join(chunk_sizes, alg_full_compaction, chunk_factor, num_join, print_log),
        "Sort Compaction": simulate_join(chunk_sizes, alg_sort_compaction, chunk_factor, num_join, print_log),
        "Binary Compaction": simulate_join(chunk_sizes, alg_binary_compaction, chunk_factor, num_join, print_log), 
        "Dynamic Compaction": simulate_join(chunk_sizes, alg_dynamic_compaction, chunk_factor, num_join, print_log),
    }

    print_color(f"-------------------------", 'green')

    for grade in grades:
        prc_cost = grades[grade][0]/1e6
        cpt_cost = grades[grade][1]/1e6
        print_color(f"[{grade}]", 'green')
        print(f"\t Total Cost: {prc_cost + cpt_cost:.2f}s\tCompute Cost: {prc_cost:.2f}s\t Compaction Cost: {cpt_cost:.2f}s")

In [19]:
chunk_sizes = generate_chunk_sizes(n=int(2e7 / 2048), mean=2048, scale=128)
chunk_factor = 16
num_join = 3
print_log = False

let_us_join(chunk_sizes, chunk_factor, num_join, print_log)

Compaction Threshold: 1024
Compaction Threshold: 69
Compaction Threshold: 0
-------------------------
[No Compaction]
	 Total Cost: 11.66s	Compute Cost: 11.66s	 Compaction Cost: 0.00s
[Full Compaction]
	 Total Cost: 7.09s	Compute Cost: 5.33s	 Compaction Cost: 1.76s
[Sort Compaction]
	 Total Cost: 6.27s	Compute Cost: 5.68s	 Compaction Cost: 0.59s
[Binary Compaction]
	 Total Cost: 6.96s	Compute Cost: 5.52s	 Compaction Cost: 1.45s
[Dynamic Compaction]
	 Total Cost: 6.55s	Compute Cost: 5.62s	 Compaction Cost: 0.94s


In [20]:
k_cpt_per_tuple_cost *= 10
chunk_sizes = generate_chunk_sizes(n=int(2e7 / 2048), mean=2048, scale=128)
chunk_factor = 16
num_join = 3
print_log = False

let_us_join(chunk_sizes, chunk_factor, num_join, print_log)

k_cpt_per_tuple_cost /= 10

Compaction Threshold: 134
Compaction Threshold: 6
Compaction Threshold: 0
-------------------------
[No Compaction]
	 Total Cost: 11.66s	Compute Cost: 11.66s	 Compaction Cost: 0.00s
[Full Compaction]
	 Total Cost: 22.89s	Compute Cost: 5.33s	 Compaction Cost: 17.56s
[Sort Compaction]
	 Total Cost: 11.53s	Compute Cost: 5.68s	 Compaction Cost: 5.85s
[Binary Compaction]
	 Total Cost: 19.89s	Compute Cost: 5.52s	 Compaction Cost: 14.36s
[Dynamic Compaction]
	 Total Cost: 11.54s	Compute Cost: 5.69s	 Compaction Cost: 5.85s


In [18]:
k_cpt_per_tuple_cost *= 10
chunk_sizes = generate_chunk_sizes(n=int(2e7 / 2048), mean=2048, scale=128)
chunk_factor = 16
num_join = 4
print_log = False

let_us_join(chunk_sizes, chunk_factor, num_join, print_log)

k_cpt_per_tuple_cost /= 10

Compaction Threshold: 1024
Compaction Threshold: 134
Compaction Threshold: 6
Compaction Threshold: 0
-------------------------
[No Compaction]
	 Total Cost: 60.20s	Compute Cost: 60.20s	 Compaction Cost: 0.00s
[Full Compaction]
	 Total Cost: 30.51s	Compute Cost: 7.11s	 Compaction Cost: 23.40s
[Sort Compaction]
	 Total Cost: 19.16s	Compute Cost: 7.46s	 Compaction Cost: 11.70s
[Binary Compaction]
	 Total Cost: 27.53s	Compute Cost: 7.30s	 Compaction Cost: 20.23s
[Dynamic Compaction]
	 Total Cost: 19.20s	Compute Cost: 7.49s	 Compaction Cost: 11.70s
